# Clustering sobre seeds dataset

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=156415ea2ec819550918fa3c08a0bf56a7e030a6dd06fc9696809607be7bb652
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [147]:
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import corr
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import length
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
import numpy as np
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import NGram
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [7]:
dataset = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/seeds_dataset.csv",header=True,inferSchema=True)

In [10]:
dataset.show(5)

+-----+---------+-----------+------------------+------------------+---------------------+----------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+----------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|            5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|           4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|           4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|           4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|           5.175|
+-----+---------+-----------+------------------+------------------+---------------------+----------------+
only showing top 5 rows



In [11]:
dataset.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

In [12]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [13]:
# Crear assembler
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [14]:
final_data = vec_assembler.transform(dataset)

In [15]:
# Escalar datos
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [16]:
scalerModel = scaler.fit(final_data)

In [17]:
final_data = scalerModel.transform(final_data)

In [18]:
# Crear kmeans
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [20]:
# Obtener los centros
centers = model.clusterCenters()

# Asignar cada punto a su clúster más cercano
predictions = model.transform(final_data)

# Calcular el error cuadrado para cada punto
def squared_distance(point, center):
    return sum((p - c) ** 2 for p, c in zip(point, center))

# Crear una nueva columna con los errores cuadrados
wssse = predictions.rdd.map(lambda row: squared_distance(row['features'], centers[row['prediction']])).sum()

print(f"Suma de los errores al cuadrado = {wssse}")

Suma de los errores al cuadrado = 319446.85125725897


In [21]:
# Ver los centros
for center in centers:
    print(center)

[ 4.06105916 10.13979506 35.80536984 11.82133095  7.50395937  3.27184732
 10.42126018]
[ 6.31670546 12.37109759 37.39491396 13.91155062  9.748067    2.39849968
 12.2661748 ]
[ 4.87257659 10.88120146 37.27692543 12.3410157   8.55443412  1.81649011
 10.32998598]


In [22]:
# Ver las predicciones
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows



# Clustering sobre hack data

In [23]:
dataset = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/hack_data.csv",header=True,inferSchema=True)

In [24]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [25]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [26]:
# Crear assembler
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [27]:
final_data = vec_assembler.transform(dataset)

In [28]:
# Escalar los datos
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [29]:
scalerModel = scaler.fit(final_data)

In [30]:
cluster_final_data = scalerModel.transform(final_data)

In [33]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    # Crear una nueva columna con los errores cuadrados
    wssse = predictions.rdd.map(lambda row: squared_distance(row['features'], centers[row['prediction']])).sum()
    print("Con K={}".format(k))
    print("Suma errores al cuadrado = " + str(wssse))
    print('--'*30)

Con K=2
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------
Con K=3
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------
Con K=4
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------
Con K=5
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------
Con K=6
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------
Con K=7
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------
Con K=8
Suma errores al cuadrado = 319446.85125725897
------------------------------------------------------------


# Regresión lineal ecommerce

In [35]:
data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [36]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [37]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App",
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [38]:
# Vector con los datos
output = assembler.transform(data)

In [39]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [40]:
final_data = output.select("features",'Yearly Amount Spent')

In [41]:
# Separar los datos
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [42]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                328|
|   mean|  500.5294314688369|
| stddev|  78.28494747129923|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [43]:
# Crear regresión
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [44]:
lrModel = lr.fit(train_data,)

In [45]:
# Ver coeficientes e intercepto
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.47790373915074,38.85406011003062,0.07445967311042413,61.55790722280369] Intercept: -1031.5203280112949


In [46]:
test_results = lrModel.evaluate(test_data)

In [48]:
# Ver residuos
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -6.0453830848128405|
|-0.23402264084307944|
| -17.514712022882918|
|  -4.461554403212119|
|    3.60676838484153|
|   -4.37765238704651|
|  -7.314716004392608|
|  3.7745467455902713|
|   -6.86180029149358|
|   1.920534952849323|
|   7.563293464368769|
|  -9.458806667726265|
|  -8.998180158707726|
|  -17.41027255359137|
| -2.3438732575622794|
| -13.907305564265243|
|  5.4800025429987045|
|   5.230859148333707|
|  15.828788177019703|
|  23.199167089056857|
+--------------------+
only showing top 20 rows



In [49]:
unlabeled_data = test_data.select('features')

In [50]:
predictions = lrModel.transform(unlabeled_data)

In [51]:
# Hacer predicciones
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|288.51662880472736|
|[30.5743636841713...|442.29843639890873|
|[30.8162006488763...| 283.6010529713519|
|[30.8794843441274...| 494.6681543880668|
|[31.3662121671876...| 426.9821141716434|
|[31.4252268808548...| 535.1443710418084|
|[31.4474464941278...| 425.9174580996166|
|[31.5316044825729...| 432.7410589837723|
|[31.7242025238451...| 510.2496875794541|
|[31.7366356860502...| 495.0129113026826|
|[31.8209982016720...| 417.1119875488446|
|[31.8279790554652...| 449.4615542146678|
|[31.8854062999117...|399.10145313118323|
|[31.9048571310136...| 491.3601299764075|
|[31.9120759292006...|   389.87858956327|
|[31.9365486184489...|441.10669045959344|
|[32.0305497162129...| 588.7944808756131|
|[32.0478009788678...|508.21971203776275|
|[32.0478146331398...| 481.5607695818237|
|[32.0498393904573...|455.52018978515844|
+--------------------+------------

In [52]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.612328222202251
MSE: 112.6215102957504


# Regresion lineal cruceros

In [56]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/cruise_ship_info.csv',inferSchema=True,header=True)

In [57]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [58]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       NULL|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     NULL|       NULL| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [59]:
# Agrupando por linea
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [61]:
# Convertir a número
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
only showing top 5 rows



In [62]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [63]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

In [64]:
output = assembler.transform(indexed)

In [65]:
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [66]:
final_data = output.select("features", "crew")

In [67]:
# Separar datos
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [68]:
# Crear regresión lineal
lr = LinearRegression(labelCol='crew')

In [69]:
lrModel = lr.fit(train_data)

In [70]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.012851102851705954,0.0024079073568636644,-0.13788474335138481,0.4275974217304414,0.8878069456843056,-0.0036786719197472884,0.0443648189421419] Intercept: -1.0053476134425352


In [71]:
test_results = lrModel.evaluate(test_data)

In [72]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 0.7149121337005412
MSE: 0.5110993589122605
R2: 0.9496830869598495


In [73]:
# Correlacción entre variables
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [74]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



# Regresion logística churn

In [75]:
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/customer_churn.csv',inferSchema=True,
                     header=True)

In [76]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [77]:
# Crear assembler
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [78]:
output = assembler.transform(data)

In [79]:
final_data = output.select('features','churn')

In [80]:
# Separar datos
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [81]:
lr_churn = LogisticRegression(labelCol='churn')

In [82]:
fitted_churn_model = lr_churn.fit(train_churn)

In [83]:
training_sum = fitted_churn_model.summary

In [84]:
training_sum.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              churn|        prediction|
+-------+-------------------+------------------+
|  count|                597|               597|
|   mean|0.17755443886097153|0.1541038525963149|
| stddev|0.38245767735486724|0.3613510385887378|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [85]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [87]:
# Ver las predicciones
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.14302479542239...|[0.53569537060862...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.84249010766883...|[0.99893369617566...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.24703952878957...|[0.77678696875800...|       0.0|
|[28.0,11204.23,0....|    0|[1.82600777643292...|[0.86128545219060...|       0.0|
|[29.0,5900.78,1.0...|    0|[3.84775471250605...|[0.97911779737878...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.48787959362432...|[0.97034093260482...|       0.0|
|[30.0,10183.98,1....|    0|[2.84700057055805...|[0.94516343121120...|       0.0|
|[30.0,12788.37,0....|    0|[2.83327725816649...|[0.94444779786731...|       0.0|
|[30.0,13473.35,0....|    0|[3.10032825515383...|[0.95690628323161...|       0.0|
|[31.0,5304.6,0.

In [88]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [89]:
# Obtener auc
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [90]:
auc

0.6833099333099334

In [91]:
final_lr_model = lr_churn.fit(final_data)

In [95]:
new_customers = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/new_customers.csv',inferSchema=True,
                              header=True)

In [96]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [97]:
test_new_customers = assembler.transform(new_customers)

In [98]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [99]:
final_results = final_lr_model.transform(test_new_customers)

In [100]:
# Prediciones sobre los nuevos datos
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



# Regresion logística titanic

In [101]:
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/titanic.csv',inferSchema=True,header=True)

In [102]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [103]:
# Seleccionar columnas
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [104]:
# Eliminar nulos
my_final_data = my_cols.na.drop()

In [105]:
# Convertirlo a one hot
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [106]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [107]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [108]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [109]:
# Juntar los vectores creados
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [110]:
# Separar datos
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [111]:
fit_model = pipeline.fit(train_titanic_data)

In [112]:
results = fit_model.transform(test_titanic_data)

In [113]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [114]:
# Predicciones
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [115]:
# Calcular AUC
AUC = my_eval.evaluate(results)

In [116]:
AUC

0.7468297101449276

# Nlp mensajes

In [117]:
data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [118]:
# Renombrar columna
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [119]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [121]:
# Añadir columna con la longitud
data = data.withColumn('length',length(data['text']))

In [122]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [123]:
# Agrupar y ver longitud media
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [124]:

# convierte una columna de texto en una lista de palabras individuales o tokens
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
# elimina las palabras vacías  como y, de, del
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
# convierte la lista de tokens en un vector de características de frecuencia de términos
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
# ajusta las frecuencias de los términos calculadas  segun su importancia
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
# convierte las etiquetas de clase de texto  en índices numéricos
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [125]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [126]:
# Usar naive bayes
nb = NaiveBayes()

In [127]:
# Utilizar los elementos anteriores
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [128]:
cleaner = data_prep_pipe.fit(data)

In [129]:
clean_data = cleaner.transform(data)

In [130]:
clean_data = clean_data.select(['label','features'])

In [131]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [132]:
# Separar los datos
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [133]:
spam_predictor = nb.fit(training)

In [134]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [135]:
# Realizar predicciones
test_results = spam_predictor.transform(testing)

In [136]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-623.85449816099...|[0.99999999981571...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1000.9023951051...|[1.0,8.0644021960...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1380.1234344522...|[1.0,2.4611867767...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-218.10549432254...|[1.0,6.2024831222...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-684.49958831074...|[1.0,2.3782514592...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-687.00741306734...|[1.0,5.4727349243...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-967.40473746258...|[1.0,4.6772902861...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1304.4256012113...|[1.0,4.5385417032...|       0.0|
|  0.0|(13424,[0,1,27,85...|[-1522.6774187203...|[0.00190656360542...|       1.0|
|  0.0|(13424,[0

In [137]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Exactitud: {}".format(acc))

Exactitud: 0.9313272650559551


# Herramientas npl

In [138]:
sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [139]:
sentenceDataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [140]:
# Crear un Tokenizer simple que divide el texto en palabras utilizando espacios en blanco como delimitador
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# Crear un RegexTokenizer que utiliza una expresión regular para tokenizar el texto.
# En este caso, se usa "\\W" (cualquier carácter que no sea una palabra) como delimitador.
regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")

# Definir una función de usuario (UDF) que cuenta el número de tokens en la lista de palabras.
# Esta función toma una lista de palabras ('words') y devuelve su longitud (el número de palabras).
countTokens = udf(lambda words: len(words), IntegerType())

# Aplicar el tokenizer simple al DataFrame llamado 'sentenceDataFrame'.
# Esto crea una nueva columna 'words' en 'tokenized' que contiene la lista de palabras tokenizadas.
tokenized = tokenizer.transform(sentenceDataFrame)

# Seleccionar las columnas 'sentence' y 'words' del DataFrame tokenizado.
# Luego, agregar una nueva columna 'tokens' que contiene el número de palabras en la lista de 'words'.
# Finalmente, mostrar el resultado sin truncar las líneas.
tokenized.select("sentence", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

# Aplicar el RegexTokenizer al DataFrame llamado 'sentenceDataFrame'.
# Esto crea una nueva columna 'words' en 'regexTokenized' con la lista de palabras tokenizadas usando la expresión regular.
regexTokenized = regexTokenizer.transform(sentenceDataFrame)

# Seleccionar las columnas 'sentence' y 'words' del DataFrame tokenizado con RegexTokenizer.
# Luego, agregar una nueva columna 'tokens' que contiene el número de palabras en la lista de 'words'.
# Finalmente, mostrar el resultado sin truncar las líneas.
regexTokenized.select("sentence", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+-----------------------------------+------------------------------------------+------+
|sentence                           |words                                     |tokens|
+-----------------------------------+------------------------------------------+------+
|Hi I heard about Spark             |[hi, i, heard, about, spark]              |5     |
|I wish Java could use case classes |[i, wish, java, could, use, case, classes]|7     |
|Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |1     |
+-----------------------------------+------------------------------------------+------+

+-----------------------------------+------------------------------------------+------+
|sentence                           |words                                     |tokens|
+-----------------------------------+------------------------------------------+------+
|Hi I heard about Spark             |[hi, i, heard, about, spark]              |5     |
|I wish Java could use case cla

In [141]:

sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])

# Elimina palabras vacías
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")
remover.transform(sentenceData).show(truncate=False)


+---+----------------------------+--------------------+
|id |raw                         |filtered            |
+---+----------------------------+--------------------+
|0  |[I, saw, the, red, balloon] |[saw, red, balloon] |
|1  |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+



In [142]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

#  Genera secuencias consecutivas de palabras
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

ngramDataFrame = ngram.transform(wordDataFrame)
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [143]:

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [144]:
# divide texto en palabras individuales
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [145]:
# convierte un conjunto de palabras (tokens) en un vector de características
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

# ajusta las frecuencias de los términos en un conjunto de datos
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[6,8,13,16],[...|
|  0.0|(20,[0,2,7,13,15,...|
|  1.0|(20,[3,4,6,11,19]...|
+-----+--------------------+



In [146]:

df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

# convierte una colección de documentos en una matriz de características de frecuencia de términos
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

model = cv.fit(df)

result = model.transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+



# Bosques en datos del colegio

In [171]:
# Load training data
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/College.csv',inferSchema=True,header=True)

In [172]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [173]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

In [174]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [175]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [177]:
# Crear vector con los datos
output = assembler.transform(data)

In [178]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [179]:
final_data = output_fixed.select("features",'PrivateIndex')

In [180]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [181]:
# Crear los 3 modelos

dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [182]:
# Entrenarlos
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [185]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [186]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [187]:
# Evaluar los modelos
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [188]:

print('Exactitud decision tree: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('Exactitud random forest: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('Exactitud GBT: {0:2.2f}%'.format(gbt_acc*100))

Exactitud decision tree: 85.71%
--------------------------------------------------------------------------------
Exactitud random forest: 92.28%
--------------------------------------------------------------------------------
Exactitud GBT: 86.10%


# Bosques en comida de perros

In [189]:
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/dog_food.csv',inferSchema=True,header=True)

In [190]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [191]:
data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [192]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [193]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [194]:
# Crear vector
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [195]:
output = assembler.transform(data)

In [196]:
rfc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')

In [197]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [198]:
final_data = output.select('features','Spoiled')
final_data.head()

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [199]:
rfc_model = rfc.fit(final_data)

In [200]:
# Importancia de las variables
rfc_model.featureImportances

SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})

## Recomendador als películas

In [213]:
from pyspark.ml.recommendation import ALS
ratings_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/proyecto-pyspark/datos/ratings.csv', header=True, inferSchema=True,sep=";")

In [214]:
ratings_df.show()

+-------+--------+------+---------+-----------+------------+
|user_id|movie_id|rating|timestamp|user_emb_id|movie_emb_id|
+-------+--------+------+---------+-----------+------------+
|      1|    1193|     5|978300760|          0|        1192|
|      1|     661|     3|978302109|          0|         660|
|      1|     914|     3|978301968|          0|         913|
|      1|    3408|     4|978300275|          0|        3407|
|      1|    2355|     5|978824291|          0|        2354|
|      1|    1197|     3|978302268|          0|        1196|
|      1|    1287|     5|978302039|          0|        1286|
|      1|    2804|     5|978300719|          0|        2803|
|      1|     594|     4|978302268|          0|         593|
|      1|     919|     4|978301368|          0|         918|
|      1|     595|     5|978824268|          0|         594|
|      1|     938|     4|978301752|          0|         937|
|      1|    2398|     4|978302281|          0|        2397|
|      1|    2918|     4

In [216]:
# Renombrar las columnas si es necesario
ratings_df = ratings_df.select(
    col('user_id').alias('user'),
    col('movie_id').alias('item'),
    col('rating')
)

In [217]:
# Configurar el modelo ALS
als = ALS(
    maxIter=10,
    regParam=0.01,
    rank=10,
    userCol="user",
    itemCol="item",
    ratingCol="rating",
    coldStartStrategy="drop"  # Para manejar datos que no están en el entrenamiento
)

# Entrenar el modelo
model = als.fit(ratings_df)

In [218]:
# Hacer recomendaciones para todos los usuarios
user_recs = model.recommendForAllUsers(10)  # 10 es el número de recomendaciones por usuario

user_recs.show(truncate=False)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user|recommendations                                                                                                                                                                          |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1   |[{3430, 7.177402}, {3074, 6.969875}, {3844, 6.5173497}, {2236, 6.3746395}, {2745, 6.256547}, {2269, 6.193067}, {454, 6.1765366}, {950, 6.1594796}, {1688, 6.136208}, {3606, 6.1069827}]  |
|2   |[{30, 8.303505}, {3502, 8.242599}, {1966, 8.227169}, {2253, 8.040067}, {3324, 7.5482717}, {1354, 7.383497}, {3823, 7.2855325}, {3145, 7.1841297}, {3194, 7.0272427}, {1277, 7.0240993}]  |
|3   |[{537, 9.206983}, {533, 9.197

In [219]:
# Hacer recomendaciones para un usuario específico
user_id = 1
user_df = spark.createDataFrame([(user_id,)], ['user'])

user_recs = model.recommendForUserSubset(user_df, 10)  # 10 es el número de recomendaciones

user_recs.show(truncate=False)

+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user|recommendations                                                                                                                                                                        |
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1   |[{3430, 7.177402}, {3074, 6.969875}, {3844, 6.5173497}, {2236, 6.3746395}, {2745, 6.256547}, {2269, 6.193067}, {454, 6.1765366}, {950, 6.1594796}, {1688, 6.136208}, {3606, 6.1069827}]|
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [220]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit

# Dividir los datos en entrenamiento y prueba
(training_data, test_data) = ratings_df.randomSplit([0.8, 0.2])

# Entrenar el modelo en los datos de entrenamiento
model = als.fit(training_data)

# Hacer predicciones sobre los datos de prueba
predictions = model.transform(test_data)

# Evaluar el modelo
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"Root Mean Square Error (RMSE) = {rmse}")


Root Mean Square Error (RMSE) = 1.2447993420339065
